# Redshift Example

https://docs.aws.amazon.com/ja_jp/redshift/latest/dg/tutorial-loading-data.html

## Definitions

In [1]:
import psycopg2 as pg
import pandas as pd

# my secret file
import secrets

In [2]:
dbname = 'test'
host = 'cluster1.cgxlfjoyvm40.us-west-2.redshift.amazonaws.com'
port = 5439
user = 'redshift'
password = secrets.password

# set you bucket name and role for access the bucket
s3_bucket_name = 's3://midaisuk-redshift-public/example-copy'
s3_iam_role = 'arn:aws:iam::441877424058:role/Redshift-s3-read'

## Connect

In [3]:
conn = pg.connect(host=host, port=port, user=user, password=password, dbname=dbname)
cur = conn.cursor()

In [4]:
pd.read_sql(sql='select version();', con=conn)

,version
0,"PostgreSQL 8.0.2 on i686-pc-linux-gnu, compile..."


## Create Tables

### reset table condition

In [5]:
sql = """
drop table if exists part cascade;
drop table if exists supplier;
drop table if exists customer;
drop table if exists dwdate;
drop table if exists lineorder;
"""
cur.execute(sql)

### create

In [6]:
sql = """
CREATE TABLE part 
(
  p_partkey     INTEGER NOT NULL,
  p_name        VARCHAR(22) NOT NULL,
  p_mfgr        VARCHAR(6),
  p_category    VARCHAR(7) NOT NULL,
  p_brand1      VARCHAR(9) NOT NULL,
  p_color       VARCHAR(11) NOT NULL,
  p_type        VARCHAR(25) NOT NULL,
  p_size        INTEGER NOT NULL,
  p_container   VARCHAR(10) NOT NULL
);

CREATE TABLE supplier 
(
  s_suppkey   INTEGER NOT NULL,
  s_name      VARCHAR(25) NOT NULL,
  s_address   VARCHAR(25) NOT NULL,
  s_city      VARCHAR(10) NOT NULL,
  s_nation    VARCHAR(15) NOT NULL,
  s_region    VARCHAR(12) NOT NULL,
  s_phone     VARCHAR(15) NOT NULL
);

CREATE TABLE customer 
(
  c_custkey      INTEGER NOT NULL,
  c_name         VARCHAR(25) NOT NULL,
  c_address      VARCHAR(25) NOT NULL,
  c_city         VARCHAR(10) NOT NULL,
  c_nation       VARCHAR(15) NOT NULL,
  c_region       VARCHAR(12) NOT NULL,
  c_phone        VARCHAR(15) NOT NULL,
  c_mktsegment   VARCHAR(10) NOT NULL
);

CREATE TABLE dwdate 
(
  d_datekey            INTEGER NOT NULL,
  d_date               VARCHAR(19) NOT NULL,
  d_dayofweek          VARCHAR(10) NOT NULL,
  d_month              VARCHAR(10) NOT NULL,
  d_year               INTEGER NOT NULL,
  d_yearmonthnum       INTEGER NOT NULL,
  d_yearmonth          VARCHAR(8) NOT NULL,
  d_daynuminweek       INTEGER NOT NULL,
  d_daynuminmonth      INTEGER NOT NULL,
  d_daynuminyear       INTEGER NOT NULL,
  d_monthnuminyear     INTEGER NOT NULL,
  d_weeknuminyear      INTEGER NOT NULL,
  d_sellingseason      VARCHAR(13) NOT NULL,
  d_lastdayinweekfl    VARCHAR(1) NOT NULL,
  d_lastdayinmonthfl   VARCHAR(1) NOT NULL,
  d_holidayfl          VARCHAR(1) NOT NULL,
  d_weekdayfl          VARCHAR(1) NOT NULL
);

CREATE TABLE lineorder 
(
  lo_orderkey          INTEGER NOT NULL,
  lo_linenumber        INTEGER NOT NULL,
  lo_custkey           INTEGER NOT NULL,
  lo_partkey           INTEGER NOT NULL,
  lo_suppkey           INTEGER NOT NULL,
  lo_orderdate         INTEGER NOT NULL,
  lo_orderpriority     VARCHAR(15) NOT NULL,
  lo_shippriority      VARCHAR(1) NOT NULL,
  lo_quantity          INTEGER NOT NULL,
  lo_extendedprice     INTEGER NOT NULL,
  lo_ordertotalprice   INTEGER NOT NULL,
  lo_discount          INTEGER NOT NULL,
  lo_revenue           INTEGER NOT NULL,
  lo_supplycost        INTEGER NOT NULL,
  lo_tax               INTEGER NOT NULL,
  lo_commitdate        INTEGER NOT NULL,
  lo_shipmode          VARCHAR(10) NOT NULL
);
"""
cur.execute(sql)

### show table definitions

In [7]:
sql = """
select * from pg_table_def where tablename = 'part';
"""
pd.read_sql(sql=sql, con=conn)

,schemaname,tablename,column,type,encoding,distkey,sortkey,notnull
0,public,part,p_partkey,integer,lzo,False,0,True
1,public,part,p_name,character varying(22),lzo,False,0,True
2,public,part,p_mfgr,character varying(6),lzo,False,0,False
3,public,part,p_category,character varying(7),lzo,False,0,True
4,public,part,p_brand1,character varying(9),lzo,False,0,True
5,public,part,p_color,character varying(11),lzo,False,0,True
6,public,part,p_type,character varying(25),lzo,False,0,True
7,public,part,p_size,integer,lzo,False,0,True
8,public,part,p_container,character varying(10),lzo,False,0,True


## import data from S3

### part table

In [8]:
sql = """
copy part from '%s/part-csv.tbl' credentials 'aws_iam_role=%s' csv null as '\\000';
"""
cur.execute(sql % (s3_bucket_name, s3_iam_role))

In [9]:
sql = """
select p_partkey, p_name, p_mfgr, p_category from part where p_mfgr is null;
"""
pd.read_sql(sql=sql, con=conn)

,p_partkey,p_name,p_mfgr,p_category
0,15,NUL next,None,MFGR#47
1,81,NUL next,None,MFGR#23
2,133,NUL next,None,MFGR#44


In [10]:
sql = """
select * from part limit 10;
"""
pd.read_sql(sql=sql, con=conn)

,p_partkey,p_name,p_mfgr,p_category,p_brand1,p_color,p_type,p_size,p_container
0,22,floral beige,MFGR#4,MFGR#44,MFGR#4421,medium,"PROMO, POLISHED BRASS",19,LG DRUM
1,24,dim white,MFGR#4,MFGR#45,MFGR#459,saddle,"MEDIUM , ""PLATED"" STEEL",20,MED CASE
2,28,honeydew dim,MFGR#2,MFGR#21,MFGR#2130,green,"SMALL, ""PLATED"" COPPER",19,JUMBO PKG
3,47,firebrick red,MFGR#2,MFGR#23,MFGR#2334,sky,LARGE BURNISHED BRASS,14,JUMBO PACK
4,83,green deep,MFGR#1,MFGR#15,MFGR#1514,chocolate,PROMO BURNISHED NICKEL,47,SM CAN
5,115,hot sienna,MFGR#1,MFGR#15,MFGR#1533,spring,STANDARD POLISHED STEEL,24,MED CAN
6,133,NUL next,None,MFGR#44,MFGR#4421,navajo,SMALL BRUSHED NICKEL,19,LG PKG
7,168,frosted brown,MFGR#3,MFGR#32,MFGR#3221,lace,SMALL BRUSHED COPPER,20,JUMBO DRUM
8,174,pink medium,MFGR#3,MFGR#35,MFGR#3539,plum,ECONOMY BRUSHED COPPER,25,LG CASE
9,197,turquoise cornsilk,MFGR#3,MFGR#31,MFGR#3113,blue,SMALL ANODIZED COPPER,18,SM JAR


### supplier table

In [11]:
sql = """
copy supplier from 's3://awssampledbuswest2/ssbgz/supplier.tbl' 
credentials 'aws_iam_role=%s' 
delimiter '|' 
gzip
region 'us-west-2';
"""
cur.execute(sql % s3_iam_role)

In [12]:
sql = """
select * from supplier limit 10;
"""
pd.read_sql(sql=sql, con=conn)

,s_suppkey,s_name,s_address,s_city,s_nation,s_region,s_phone
0,2,Supplier#000000002,89eJ5ksX3Imx,ETHIOPIA 1,ETHIOPIA,AFRICA,15-768-687-3665
1,4,Supplier#000000004,Bk7ah4CK8SYQTep,MOROCCO 4,MOROCCO,AFRICA,25-128-190-5944
2,6,Supplier#000000006,tQxuVm7,KENYA 2,KENYA,AFRICA,24-114-968-4951
3,8,Supplier#000000008,9Sq4bBH2FQEmaFOocY,PERU 6,PERU,AMERICA,27-147-574-9335
4,10,Supplier#000000010,Saygah3gYWM,UNITED ST9,UNITED STATES,AMERICA,34-741-346-9870
5,12,Supplier#000000012,aLIW,INDIA 5,INDIA,ASIA,18-791-276-1263
6,14,Supplier#000000014,EXsnO5pTN,MOROCCO 0,MOROCCO,AFRICA,25-845-129-3851
7,16,Supplier#000000016,YjP5C55zHDXL7LalK27zfQ,RUSSIA 5,RUSSIA,EUROPE,32-781-609-3107
8,18,Supplier#000000018,PGGVE5PWA,MOZAMBIQU0,MOZAMBIQUE,AFRICA,26-155-215-1315
9,20,Supplier#000000020,"iybAE,RmTymrZVY",CANADA 0,CANADA,AMERICA,13-957-234-8742


## compare copy time

In [13]:
%%time
sql = """
copy lineorder from 's3://awssampledb/load/lo/lineorder-single.tbl' 
credentials 'aws_iam_role=%s'
gzip
compupdate off
region 'us-east-1';
"""
cur.execute(sql % s3_iam_role)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1min 5s


In [14]:
%%time
sql = """
copy lineorder from 's3://awssampledb/load/lo/lineorder-multi.tbl' 
credentials 'aws_iam_role=%s'
gzip
compupdate off
region 'us-east-1';
"""
cur.execute(sql % s3_iam_role)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 51.8 s


*In case of single node cluster, there are no significant difference between single and multi file.*

## show compression type

In [15]:
sql = """
select "column", type, encoding from pg_table_def where tablename = 'part';
"""
pd.read_sql(sql=sql, con=conn)

,column,type,encoding
0,p_partkey,integer,delta
1,p_name,character varying(22),text255
2,p_mfgr,character varying(6),bytedict
3,p_category,character varying(7),bytedict
4,p_brand1,character varying(9),lzo
5,p_color,character varying(11),bytedict
6,p_type,character varying(25),bytedict
7,p_size,integer,delta
8,p_container,character varying(10),bytedict


In [16]:
sql = """
select "column", type, encoding from pg_table_def where tablename = 'supplier';
"""
pd.read_sql(sql=sql, con=conn)

,column,type,encoding
0,s_suppkey,integer,delta
1,s_name,character varying(25),lzo
2,s_address,character varying(25),lzo
3,s_city,character varying(10),bytedict
4,s_nation,character varying(15),bytedict
5,s_region,character varying(12),bytedict
6,s_phone,character varying(15),lzo
